In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

from tqdm import tqdm

import keras
import keras.backend as K
import tensorflow as tf
from keras.models import Model
from keras.utils.np_utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.losses import categorical_crossentropy
from keras.layers import Input, Dense, Dropout, GRU


from sklearn import preprocessing
le = preprocessing.LabelEncoder()

# import argparse
# import os
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_sess = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/sessions_train.csv')
df_sess['prev_items'] = df_sess["prev_items"].apply(lambda x: x.strip("[]").split(" "))
df_sess['next_item'] = df_sess["next_item"].apply(lambda x: x.strip("[]").split(" "))

# df_prod = pd.read_csv('products_train.csv')

df_sess

,prev_items,next_item,locale
0,"['B09W9FND7K', 'B09JSPLN1M']",[B09M7GY217],DE
1,"['B076THCGSG', 'B007MO8IME', 'B08MF65MLV', 'B0...",[B001B4THSA],DE
2,"['B0B1LGXWDS', 'B00AZYORS2', 'B0B1LGXWDS', 'B0...",[B0767DTG2Q],DE
3,"['B09XMTWDVT', 'B0B4MZZ8MB', 'B0B7HZ2GWX', 'B0...",[B0B4R9NN4B],DE
4,"['B09Y5CSL3T', 'B09Y5DPTXN', 'B09FKD61R8']",[B0BGVBKWGZ],DE
...,...,...,...
3606244,"['B086CYFSKW', 'B0874F9859', 'B086CYFSKW']",[B07B5TYD76],IT
3606245,"['B09NRZKZ7V', 'B08WJTPV93']",[B08L1P4C3D],IT
3606246,"['B085JFX7MP', 'B085JGHW8R']",[B01MPWVD44],IT
3606247,"['B00B0UING2', 'B00B0UING2']",[B00D3HYEZ4],IT


In [ ]:
df_sess_task1 = df_sess[df_sess['locale'].isin(['DE', 'UK', 'JP'])]
df_sess_task1

,prev_items,next_item,locale
0,"['B09W9FND7K', 'B09JSPLN1M']",[B09M7GY217],DE
1,"['B076THCGSG', 'B007MO8IME', 'B08MF65MLV', 'B0...",[B001B4THSA],DE
2,"['B0B1LGXWDS', 'B00AZYORS2', 'B0B1LGXWDS', 'B0...",[B0767DTG2Q],DE
3,"['B09XMTWDVT', 'B0B4MZZ8MB', 'B0B7HZ2GWX', 'B0...",[B0B4R9NN4B],DE
4,"['B09Y5CSL3T', 'B09Y5DPTXN', 'B09FKD61R8']",[B0BGVBKWGZ],DE
...,...,...,...
3272711,"['B06XK89969', 'B01NGT5NF4', 'B00D5Z89C8', 'B0...",[B07VL2W1DR],UK
3272712,"['B076M85W1K', 'B07L8792Q9', 'B095RW318L', 'B0...",[B095RQ2LCY],UK
3272713,"['B00JQDIQRQ', 'B001O59QQE']",[B088M5YT6Y],UK
3272714,"['B07QMHMLJZ', 'B07FPYYMC4']",[B07PFF814D],UK


In [ ]:
df_sess_task2 = df_sess[df_sess['locale'].isin(['ES', 'FR', 'IT'])]
df_sess_task2 = df_sess_task2.drop(columns=['locale'])

df_sess_task2

,prev_items,next_item
3272716,"['B08MV5B53K', 'B08MV4RCQR', 'B08MV5B53K']",[B012408XPC]
3272717,"['B07JGW4QWX', 'B085VCXHXL']",[B07JFPYN5P]
3272718,"['B08BFQ52PR', 'B08LVSTZVF', 'B08BFQ52PR']",[B08NJP3KT6]
3272719,"['B08PPBF9C6', 'B08PPBF9C6', 'B08PPBF9C6', 'B0...",[B08PP6BLLK]
3272720,"['B0B6W67XCR', 'B0B712FY2M', 'B0B6ZYJ3S2']",[B09SL4MBM2]
...,...,...
3606244,"['B086CYFSKW', 'B0874F9859', 'B086CYFSKW']",[B07B5TYD76]
3606245,"['B09NRZKZ7V', 'B08WJTPV93']",[B08L1P4C3D]
3606246,"['B085JFX7MP', 'B085JGHW8R']",[B01MPWVD44]
3606247,"['B00B0UING2', 'B00B0UING2']",[B00D3HYEZ4]


In [ ]:
df_train = df_sess_task2.copy()

df_train['next_item'] = df_train.apply(lambda row: ["'" + x + "'" for x in row['next_item']], axis=1)
df_train['prev_items'] = df_train.apply(lambda row: row['prev_items'] + row['next_item'], axis=1)

df_train = df_train.drop(['next_item'], axis=1) # change this to append it to prev_items column

df_train = df_train.rename(columns={"index":"SessionId"})
df_train['SessionId'] = df_train.index

df_train = df_train.explode('prev_items')

df_train = df_train.rename(columns={"prev_items":"ItemId"})

df_train

,ItemId,SessionId
3272716,'B08MV5B53K',3272716
3272716,'B08MV4RCQR',3272716
3272716,'B08MV5B53K',3272716
3272716,'B012408XPC',3272716
3272717,'B07JGW4QWX',3272717
...,...,...
3606248,'B092S9D1SD',3606248
3606248,'B09XQQ1S72',3606248
3606248,'B0852MS7QC',3606248
3606248,'B0B1V43MN1',3606248


In [ ]:
df_train['ItemId'] = le.fit_transform(df_train.ItemId.values)

item_supports = df_train.groupby('ItemId').size()
df_train = df_train[np.in1d(df_train.ItemId, item_supports[item_supports>=4].index)]

session_lengths = df_train.groupby('SessionId').size()
df_train = df_train[np.in1d(df_train.SessionId, session_lengths[session_lengths>1].index)]

# session_lengths = df_train.groupby('SessionId').size()
# df_train = df_train[np.in1d(df_train.SessionId, session_lengths[session_lengths>=2].index)]

df_train['ItemId'] = le.inverse_transform(df_train.ItemId)

# df_train['Time'] = df_train.groupby('SessionId').cumcount()
# df_train['Time'] += 1
df_train['Time'] = np.arange(df_train.shape[0])

# df_train['LastInSess'] = df_train["SessionId"].ne(df_train["SessionId"].shift()) # true if first
df_train['LastInSess'] = df_train['SessionId'].ne(df_train['SessionId'].shift(-1)).fillna(True)
df_train

,ItemId,SessionId,Time,LastInSess
3272716,'B08MV5B53K',3272716,0,False
3272716,'B08MV4RCQR',3272716,1,False
3272716,'B08MV5B53K',3272716,2,False
3272716,'B012408XPC',3272716,3,True
3272717,'B07JGW4QWX',3272717,4,False
...,...,...,...,...
3606248,'B092S9D1SD',3606248,1449572,False
3606248,'B09XQQ1S72',3606248,1449573,False
3606248,'B0852MS7QC',3606248,1449574,False
3606248,'B0B1V43MN1',3606248,1449575,False


In [ ]:
df_test = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/sessions_test_task2.csv')
df_test['prev_items'] = df_test["prev_items"].apply(lambda x: x.strip("[]").split(" "))

df_test

,prev_items,locale
0,"['B07GTS7SWK', 'B07GTS7SWK']",ES
1,"['B0B33YWVHR', '849988993X', 'B09QQG85HM', 'B0...",ES
2,"['B08FMPXDTJ', 'B0B4612MTM', 'B0B45YR21M', 'B0...",ES
3,"['B07R3W4XQ7', 'B07R3FB5B5']",ES
4,"['B09V4KBWPL', 'B09G9FTLPB']",ES
...,...,...
34685,"['B089F3YK6X', 'B01BBW9TEI', 'B01BBW9Q46']",IT
34686,"['B078XYC42V', 'B09TW31SND']",IT
34687,"['B07ZG3V3X2', 'B09PGKF9Y8']",IT
34688,"['B0B3JRGZLM', 'B09SHQJ8NV']",IT


In [ ]:
df_t = df_test.copy()

df_t = df_t.rename(columns={"index":"SessionId"})

df_t['SessionId'] = df_t.index

df_t = df_t.explode('prev_items')

df_t = df_t.rename(columns={"prev_items":"ItemId"})

# df_t['Time'] = df_t.groupby('SessionId').cumcount()
df_t['Time'] = np.arange(df_t.shape[0])

df_t

,ItemId,locale,SessionId,Time
0,'B07GTS7SWK',ES,0,0
0,'B07GTS7SWK',ES,0,1
1,'B0B33YWVHR',ES,1,2
1,'849988993X',ES,1,3
1,'B09QQG85HM',ES,1,4
...,...,...,...,...
34688,'B0B3JRGZLM',IT,34688,132546
34688,'B09SHQJ8NV',IT,34688,132547
34689,'B08ZHKGYXR',IT,34689,132548
34689,'B07Z8PZTGL',IT,34689,132549


In [ ]:
df_pred = df_test.copy()
df_pred['next_item_prediction'] = df_pred["prev_items"].apply(lambda x: [x[0]]) ## filler

df_pred = df_pred.rename(columns={"index":"SessionId"})
df_pred['SessionId'] = df_pred.index

df_pred

,prev_items,locale,next_item_prediction,SessionId
0,"['B07GTS7SWK', 'B07GTS7SWK']",ES,['B07GTS7SWK'],0
1,"['B0B33YWVHR', '849988993X', 'B09QQG85HM', 'B0...",ES,['B0B33YWVHR'],1
2,"['B08FMPXDTJ', 'B0B4612MTM', 'B0B45YR21M', 'B0...",ES,['B08FMPXDTJ'],2
3,"['B07R3W4XQ7', 'B07R3FB5B5']",ES,['B07R3W4XQ7'],3
4,"['B09V4KBWPL', 'B09G9FTLPB']",ES,['B09V4KBWPL'],4
...,...,...,...,...
34685,"['B089F3YK6X', 'B01BBW9TEI', 'B01BBW9Q46']",IT,['B089F3YK6X'],34685
34686,"['B078XYC42V', 'B09TW31SND']",IT,['B078XYC42V'],34686
34687,"['B07ZG3V3X2', 'B09PGKF9Y8']",IT,['B07ZG3V3X2'],34687
34688,"['B0B3JRGZLM', 'B09SHQJ8NV']",IT,['B0B3JRGZLM'],34688


In [ ]:
# Step 2: Define the split ratios
train_ratio = 0.9
dev_ratio = 0.1
# test_ratio = 0.02

# Step 3: Calculate the sizes of each set
total_rows = len(df_train)
train_size = int(total_rows * train_ratio)
dev_size = int(total_rows * dev_ratio)
test_size = total_rows - train_size - dev_size

# Step 4: Split the dataframe based on the calculated sizes
train_df = df_train[:train_size]
dev_df = df_train[train_size:train_size + dev_size]
# test_df = df_train[train_size + dev_size:]
test_df = df_t

# Step 5: Verify the sizes of each set
print("Train set size:", len(train_df))
print("Dev set size:", len(dev_df))
print("Test set size:", len(test_df))

train_df.to_csv('train.csv', index=False)
dev_df.to_csv('dev.csv', index=False)
test_df.to_csv('test.csv', index=False)

Train set size: 1304619
Dev set size: 144957
Test set size: 132551


In [ ]:
class SessionDataset:
    """Credit to yhs-968/pyGRU4REC."""
    def __init__(self, data, sep='\t', session_key='SessionId', item_key='ItemId', time_key='Time', n_samples=-1, itemmap=None, time_sort=False):
        """
        Args:
            path: path of the csv file
            sep: separator for the csv
            session_key, item_key, time_key: name of the fields corresponding to the sessions, items, time
            n_samples: the number of samples to use. If -1, use the whole dataset.
            itemmap: mapping between item IDs and item indices
            time_sort: whether to sort the sessions by time or not
        """
        self.df = data
        self.session_key = session_key
        self.item_key = item_key
        self.time_key = time_key
        self.time_sort = time_sort
        self.add_item_indices(itemmap=itemmap)
        self.df.sort_values([session_key, time_key], inplace=True)

        # Sort the df by time, and then by session ID. That is, df is sorted by session ID and
        # clicks within a session are next to each other, where the clicks within a session are time-ordered.

        self.click_offsets = self.get_click_offsets()
        self.session_idx_arr = self.order_session_idx()

    def get_click_offsets(self):
        """
        Return the offsets of the beginning clicks of each session IDs,
        where the offset is calculated against the first click of the first session ID.
        """
        offsets = np.zeros(self.df[self.session_key].nunique() + 1, dtype=np.int32)
        # group & sort the df by session_key and get the offset values
        offsets[1:] = self.df.groupby(self.session_key).size().cumsum()

        return offsets

    def order_session_idx(self):
        """ Order the session indices """
        if self.time_sort:
            # starting time for each sessions, sorted by session IDs
            sessions_start_time = self.df.groupby(self.session_key)[self.time_key].min().values
            # order the session indices by session starting times
            session_idx_arr = np.argsort(sessions_start_time)
        else:
            session_idx_arr = np.arange(self.df[self.session_key].nunique())

        return session_idx_arr

    def add_item_indices(self, itemmap=None):
        """
        Add item index column named "item_idx" to the df
        Args:
            itemmap (pd.DataFrame): mapping between the item Ids and indices
        """
        if itemmap is None:
            item_ids = self.df[self.item_key].unique()  # unique item ids
            item2idx = pd.Series(data=np.arange(len(item_ids)),
                                 index=item_ids)
            itemmap = pd.DataFrame({self.item_key:item_ids,
                                   'item_idx':item2idx[item_ids].values})

        self.itemmap = itemmap
        self.df = pd.merge(self.df, self.itemmap, on=self.item_key, how='inner')
        # print(self.itemmap[self.itemmap['ItemId'] == "'B08BXBBMDM'"])

    @property
    def items(self):
        return self.itemmap.ItemId.unique()

class SessionDataLoader:
    """Credit to yhs-968/pyGRU4REC."""
    def __init__(self, dataset, batch_size=50):
        """
        A class for creating session-parallel mini-batches.
        Args:
            dataset (SessionDataset): the session dataset to generate the batches from
            batch_size (int): size of the batch
        """
        self.dataset = dataset
        self.batch_size = batch_size
        self.done_sessions_counter = 0

    def __iter__(self):
        """ Returns the iterator for producing session-parallel training mini-batches.
        Yields:
            input (B,):  Item indices that will be encoded as one-hot vectors later.
            target (B,): a Variable that stores the target item indices
            masks: Numpy array indicating the positions of the sessions to be terminated
        """

        df = self.dataset.df
        session_key='SessionId'
        item_key='ItemId'
        time_key='TimeStamp'
        self.n_items = df[item_key].nunique()+1
        click_offsets = self.dataset.click_offsets
        session_idx_arr = self.dataset.session_idx_arr

        iters = np.arange(self.batch_size)
        maxiter = iters.max()
        start = click_offsets[session_idx_arr[iters]]
        end = click_offsets[session_idx_arr[iters] + 1]
        mask = [] # indicator for the sessions to be terminated
        finished = False

        while not finished:
            minlen = (end - start).min()
            # Item indices (for embedding) for clicks where the first sessions start
            idx_target = df.item_idx.values[start]
            for i in range(minlen - 1):
                # Build inputs & targets
                idx_input = idx_target
                idx_target = df.item_idx.values[start + i + 1]
                inp = idx_input
                target = idx_target
                yield inp, target, mask

            # click indices where a particular session meets second-to-last element
            start = start + (minlen - 1)
            # see if how many sessions should terminate
            mask = np.arange(len(iters))[(end - start) <= 1]
            self.done_sessions_counter = len(mask)
            for idx in mask:
                maxiter += 1
                if maxiter >= len(click_offsets) - 1:
                    finished = True
                    break
                # update the next starting/ending point
                iters[idx] = maxiter
                start[idx] = click_offsets[session_idx_arr[maxiter]]
                end[idx] = click_offsets[session_idx_arr[maxiter] + 1]

    def test_iter(self):
        """ Returns the iterator for producing session-parallel training mini-batches.
        Yields:
            input (B,):  Item indices that will be encoded as one-hot vectors later.
            target (B,): a Variable that stores the target item indices
            masks: Numpy array indicating the positions of the sessions to be terminated
        """

        df = self.dataset.df
        # df = df.reset_index()
        # df['Time'] = df.index
        # print(df[df['SessionId'] == 875])
        # print(df.tail(10))
        session_key='SessionId'
        item_key='ItemId'
        time_key='TimeStamp'
        self.n_items = df[item_key].nunique()+1
        click_offsets = self.dataset.click_offsets
        session_idx_arr = self.dataset.session_idx_arr

        iters = np.arange(self.batch_size)
        maxiter = iters.max()
        start = click_offsets[session_idx_arr[iters]]
        end = click_offsets[session_idx_arr[iters] + 1]
        mask = [] # indicator for the sessions to be terminated
        sessions = []
        finished = False


        while not finished:
            minlen = (end - start).min()
            # Item indices (for embedding) for clicks where the first sessions start
            idx_target = df.item_idx.values[start]

            # Build inputs & targets
            idx_input = idx_target
            idx_target = df.item_idx.values[start + 1]
            sessions = df.SessionId.values[start]
            # print(df.SessionId.values[start + 1])
            inp = idx_input
            target = idx_target

            # click indices where a particular session meets second-to-last element
            start = start + minlen
            # see if how many sessions should terminate
            mask_next = np.arange(len(iters))[(end - start) == 0]

            yield inp, target, mask, sessions, mask_next

            mask = mask_next

            self.done_sessions_counter = len(mask)

            for idx in mask:
                maxiter += 1
                if maxiter >= len(click_offsets) - 1:
                    finished = True
                    break
                # update the next starting/ending point
                iters[idx] = maxiter
                start[idx] = click_offsets[session_idx_arr[maxiter]]
                end[idx] = click_offsets[session_idx_arr[maxiter] + 1]

def create_model(args):
    emb_size = 50
    hidden_units = 100
    size = emb_size

    input = Input(batch_shape=(args.batch_size, 1, args.train_n_items))
    gru, gru_states = GRU(hidden_units, stateful=True, return_state=True, name="GRU")(input)
    drop2 = Dropout(0.25)(gru)
    predictions = Dense(args.train_n_items, activation='softmax')(drop2)
    model = Model(inputs=input, outputs=[predictions])
    opt = tf.keras.optimizers.Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999)
    model.compile(loss=categorical_crossentropy, optimizer=opt)
    model.summary()

    filepath='./model_checkpoint.h5'
    checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=2, save_best_only=True, mode='min')
    callbacks_list = []
    return model


def get_metrics(model, args, train_generator_map, recall_k=20, mrr_k=20):

    test_dataset = SessionDataset(args.test_data, itemmap=train_generator_map)
    test_generator = SessionDataLoader(test_dataset, batch_size=args.batch_size).test_iter()

    n = 0
    rec_sum = 0
    mrr_sum = 0

    item_dict = dict(zip(train_generator_map['item_idx'], train_generator_map['ItemId']))

    print("Evaluating model...")
    for feat, label, finished_prev_step, sessions, finish_this_step in test_generator:
        # print(feat[2], test_dataset.itemmap.iloc[feat[2]]['ItemId'])
        # print(feat, label, sessions)

        gru_layer = model.get_layer(name="GRU")
        hidden_states = gru_layer.states[0].numpy()
        for elt in finished_prev_step:
            hidden_states[elt, :] = 0
        gru_layer.reset_states(states=hidden_states)

        target_oh = to_categorical(label, num_classes=args.train_n_items)
        input_oh  = to_categorical(feat,  num_classes=args.train_n_items)
        input_oh = np.expand_dims(input_oh, axis=1)

        pred = model.predict(input_oh, batch_size=args.batch_size)

        # print(test_dataset.itemmap.iloc[feat]['ItemId'])

        # for row_idx in range(feat.shape[0]):
        for row_idx in range(tf.shape(feat)[0]):
            pred_row = pred[row_idx]
            label_row = target_oh[row_idx]

            rec_idx =  pred_row.argsort()[-recall_k:][::-1]
            mrr_idx =  pred_row.argsort()[-mrr_k:][::-1]
            tru_idx = label_row.argsort()[-1:][::-1]
            pred_idx = pred_row.argsort()[-100:][::-1]

            if row_idx in finish_this_step:
              result = [item_dict.get(idx) for idx in pred_idx]
              cond = df_pred['SessionId'] == sessions[row_idx]
              row_idx = df_pred[cond].index.item()
              df_pred.at[row_idx, 'next_item_prediction'] = result

            n += 1

            if tru_idx[0] in rec_idx:
                rec_sum += 1

            if tru_idx[0] in mrr_idx:
                mrr_sum += 1/int((np.where(mrr_idx == tru_idx[0])[0]+1))


    recall = rec_sum/n
    mrr = mrr_sum/n
    return (recall, recall_k), (mrr, mrr_k)


def train_model(model, args):
    train_dataset = SessionDataset(args.train_data)
    model_to_train = model
    batch_size = args.batch_size

    for epoch in range(1, args.epochs):
        with tqdm(total=args.train_samples_qty) as pbar:
            loader = SessionDataLoader(train_dataset, batch_size=batch_size)
            for feat, target, mask in loader:

                gru_layer = model_to_train.get_layer(name="GRU")
                hidden_states = gru_layer.states[0].numpy()
                for elt in mask:
                    hidden_states[elt, :] = 0
                gru_layer.reset_states(states=hidden_states)

                input_oh = to_categorical(feat, num_classes=loader.n_items)
                input_oh = np.expand_dims(input_oh, axis=1)

                target_oh = to_categorical(target, num_classes=loader.n_items)

                tr_loss = model_to_train.train_on_batch(input_oh, target_oh)

                pbar.set_description("Epoch {0}. Loss: {1:.5f}".format(epoch, tr_loss))
                pbar.update(loader.done_sessions_counter)

        if args.save_weights:
            print("Saving weights...")
            model_to_train.save('./GRU4REC_{}.h5'.format(epoch))

        if args.eval_all_epochs:
            (rec, rec_k), (mrr, mrr_k) = get_metrics(model_to_train, args, train_dataset.itemmap)
            print("\t - Recall@{} epoch {}: {:5f}".format(rec_k, epoch, rec))
            print("\t - MRR@{}    epoch {}: {:5f}\n".format(mrr_k, epoch, mrr))

    if not args.eval_all_epochs:
        (rec, rec_k), (mrr, mrr_k) = get_metrics(model_to_train, args, train_dataset.itemmap)
        print("\t - Recall@{} epoch {}: {:5f}".format(rec_k, args.epochs, rec))
        print("\t - MRR@{}    epoch {}: {:5f}\n".format(mrr_k, args.epochs, mrr))



In [ ]:
class Arguments:
  def __init__(self):
    self.train_path = 'train.csv'
    self.dev_path = 'dev.csv'
    self.test_path = 'test.csv'
    self.resume = '/content/drive/MyDrive/ColabNotebooks/GRU4REC_4.h5'
    # self.resume = False
    self.eval_only = False
    self.batch_size = 512
    self.eval_all_epochs = False
    self.save_weights = True
    self.epochs = 2

args = Arguments()

# args.train_data = pd.read_csv(args.train_path, sep='\t', dtype={'ItemId': np.int64})
# args.dev_data   = pd.read_csv(args.dev_path,   sep='\t', dtype={'ItemId': np.int64})
# args.test_data  = pd.read_csv(args.test_path,  sep='\t', dtype={'ItemId': np.int64})

args.train_data = pd.read_csv(args.train_path)
args.dev_data   = pd.read_csv(args.dev_path)
args.test_data  = pd.read_csv(args.test_path)

args.train_n_items = len(args.train_data['ItemId'].unique()) + 1

args.train_samples_qty = len(args.train_data['SessionId'].unique()) + 1
args.test_samples_qty = len(args.test_data['SessionId'].unique()) + 1

if args.resume:
    try:
        model = keras.models.load_model(args.resume)
        print("Model checkpoint '{}' loaded!".format(args.resume))
    except OSError:
        print("Model checkpoint could not be loaded. Training from scratch...")
        model = create_model(args)
else:
    model = create_model(args)

if args.eval_only:
    train_dataset = SessionDataset(args.train_data)
    (rec, rec_k), (mrr, mrr_k) = get_metrics(model, args, train_dataset.itemmap)
    print("\t - Recall@{} epoch {}: {:5f}".format(rec_k, -1, rec))
    print("\t - MRR@{}    epoch {}: {:5f}\n".format(mrr_k, -1, mrr))
else:
    train_model(model, args)

In [ ]:
df_pred.to_csv('/content/drive/MyDrive/ColabNotebooks/temp.csv')

In [ ]:
def transform_values(value):
    converted_values = ','.join([item.replace("\n", "") for item in value])
    return eval(f"[{converted_values}]")

In [ ]:
df_predd = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/temp.csv')
df_predd['next_item_prediction'] = df_predd['next_item_prediction'].apply(lambda x: transform_values(eval(x)))
df_predd

,Unnamed: 0,prev_items,locale,next_item_prediction,SessionId
0,0,"[""'B07GTS7SWK'"", ""'B07GTS7SWK'""]",ES,"[B0BCR1KLCP, B00VYAWN5U, B074581R8Y, B07KRG286...",0
1,1,"[""'B0B33YWVHR'"", ""'849988993X'"", ""'B09QQG85HM'...",ES,"[B0BKSPPZXM, B07MGSW2GQ, B09Z1VH85Q, B09K7TDY1...",1
2,2,"[""'B08FMPXDTJ'"", ""'B0B4612MTM'"", ""'B0B45YR21M'...",ES,"[B0B4625TR5, B0B4612MTM, B09Q9C9QP3, B0B38J377...",2
3,3,"[""'B07R3W4XQ7'"", ""'B07R3FB5B5'""]",ES,[B07R3W4XQ7],3
4,4,"[""'B09V4KBWPL'"", ""'B09G9FTLPB'""]",ES,"[B09G9LF91K, B09G9LF91K, B09G98QYSD, B00IIOI9Q...",4
...,...,...,...,...,...
34685,34685,"[""'B089F3YK6X'"", ""'B01BBW9TEI'"", ""'B01BBW9Q46'""]",IT,[B089F3YK6X],34685
34686,34686,"[""'B078XYC42V'"", ""'B09TW31SND'""]",IT,[B078XYC42V],34686
34687,34687,"[""'B07ZG3V3X2'"", ""'B09PGKF9Y8'""]",IT,[B07ZG3V3X2],34687
34688,34688,"[""'B0B3JRGZLM'"", ""'B09SHQJ8NV'""]",IT,[B0B3JRGZLM],34688


In [ ]:
# train_dataset = SessionDataset(args.train_data)
# item_dict = dict(zip(train_dataset.itemmap['item_idx'], train_dataset.itemmap['ItemId']))
# itemmapp = train_dataset.itemmap
# fix = df_predd.copy()

In [ ]:
# input = []
# m = 0
# for i in range(len(fix)):
#   if len(fix['next_item_prediction'][i]) == 1:
#     inp = eval(fix['prev_items'][i])[len(eval(fix['prev_items'][i])) - 1]
#     res = itemmapp[itemmapp['ItemId'] == inp]['item_idx'].index
#     if len(res) != 0:
#       input.append(res[0])
#       m += 1
#       print(m)


In [ ]:
# try:
#   model = keras.models.load_model('/content/drive/MyDrive/ColabNotebooks/GRU4REC_4.h5')
#   print("Model checkpoint '{}' loaded!".format(args.resume))
# except OSError:
#   print("Model checkpoint could not be loaded. Training from scratch...")
#   model = create_model(args)

# gru_layer = model.get_layer(name="GRU")
# hidden_states = gru_layer.states[0].numpy()
# for elt in range(512):
#     hidden_states[elt, :] = 0
# gru_layer.reset_states(states=hidden_states)

# for i in range(100):
#   input.append(0)

# input_oh  = to_categorical(input,  num_classes=args.train_n_items)
# input_oh = np.expand_dims(input_oh, axis=1)
# pred = model.predict(input_oh, batch_size=args.batch_size)

# to_place = []

# for i in range(512):
#   pred_idx = pred[i].argsort()[-100:][::-1]
#   ItemIds = [item_dict.get(idx) for idx in pred_idx]
#   to_place.append(transform_values(ItemIds))

# print(len(to_place))
# result = np.vstack(to_place)
# print(result)

# m = 0
# fix = df_predd.copy()
# fix['indicator'] = False
# for i in range(len(fix)):
#   if len(fix['next_item_prediction'][i]) == 1:
#     inp = eval(fix['prev_items'][i])[len(eval(fix['prev_items'][i])) - 1]
#     res = itemmapp[itemmapp['ItemId'] == inp]['item_idx'].index
#     if len(res) != 0:
#       fix['indicator'][i] = True
#       # fix.loc[i, 'next_item_prediction'] = result[m]
#       m += 1
# print(m)

Model checkpoint '/content/drive/MyDrive/ColabNotebooks/GRU4REC_4.h5' loaded!
1/1 [==============================] - 2s 2s/step
512
[['B0BDJCNTDS' 'B0BDHR54XS' 'B0BDJ978JD' ... 'B0B5TTH9QG' 'B09SHV78TN'
  'B0B7P2FTR8']
 ['B08HPK2HQV' 'B09NT33LZN' 'B0B56NWXYH' ... 'B0B3DRZTWF' 'B09XMN5MW7'
  'B01MG62Z5M']
 ['B079HBT9LV' 'B079KD7ZZ2' 'B07DM8QD2B' ... 'B00G3C19IG' 'B079D3WY8Q'
  'B07P5NC2QF']
 ...
 ['B07ZTVDJNZ' 'B08K9BQBXB' 'B08MV4RCQR' ... 'B08HZ6HCCT' 'B07CCVMSKV'
  'B0B5TV84KK']
 ['B07ZTVDJNZ' 'B08K9BQBXB' 'B08MV4RCQR' ... 'B08HZ6HCCT' 'B07CCVMSKV'
  'B0B5TV84KK']
 ['B07ZTVDJNZ' 'B08K9BQBXB' 'B08MV4RCQR' ... 'B08HZ6HCCT' 'B07CCVMSKV'
  'B0B5TV84KK']]


<ipython-input-30-ad273762e305>:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fix['indicator'][i] = True


412


In [ ]:
# m = 0
# for i in range(len(fix)):
#   if fix['indicator'][i] == True:
#     fix.at[i, 'next_item_prediction'] = result[m]
#     m += 1
# print(m)

In [ ]:
def check_predictions(predictions):
    """
    These tests need to pass as they will also be applied on the evaluator
    """
    test_locale_names = df_test['locale'].unique()
    for locale in test_locale_names:
        sess_test = df_test.query(f'locale == "{locale}"')
        preds_locale =  predictions[predictions['locale'] == sess_test['locale'].iloc[0]]
        assert sorted(preds_locale.index.values) == sorted(sess_test.index.values), f"Session ids of {locale} doesn't match"

In [ ]:
check_predictions(df_predd)

In [ ]:
df_predd[['locale', 'next_item_prediction']].to_parquet('submission_task2.parquet', engine='pyarrow')

In [ ]:
# !cp GRU4REC_4.h5 /content/drive/MyDrive/ColabNotebooks/